In [1]:
import pandas as pd
import duckdb as dk

%load_ext sql
%sql duckdb:///data/vermont.duckdb
%config SqlMagic.named_parameters="enabled"

Connecting to 'duckdb:///data/vermont.duckdb'

In [2]:
%%sql
SELECT COUNT(*) as num_checklists
FROM (
    SELECT COUNT(*) FROM sightings_clean
    GROUP BY sampling_id
) t

Running query in 'duckdb:///data/vermont.duckdb'

num_checklists
299742


In [15]:
%%sql

CREATE OR REPLACE TABLE dask_sightings_clean AS
SELECT *
FROM 'data\VT_unique_complete_hotspot_sightings.parquet';

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [37]:
%%sql

SELECT COUNT(*) as num_group_checklists_dask
FROM (
    SELECT COUNT(*) 
    FROM dask_sightings_clean
    WHERE "GROUP IDENTIFIER" IS NOT NULL
    GROUP BY "SAMPLING EVENT IDENTIFIER"
) t

Running query in 'duckdb:///data/vermont.duckdb'

num_group_checklists_dask
92623


In [36]:
%%sql

SELECT COUNT(*) as num_group_checklists_duckdb
FROM (
    SELECT COUNT(*) 
    FROM sightings_clean
    WHERE group_id IS NOT NULL
    GROUP BY sampling_id
) t

Running query in 'duckdb:///data/vermont.duckdb'

num_group_checklists_duckdb
92538


In [38]:
%%sql
SELECT COUNT(*) as num_group_checklists_missing_from_dask
FROM (
    SELECT "GROUP IDENTIFIER"
    FROM dask_sightings_clean
    EXCEPT
    SELECT DISTINCT group_id
    FROM sightings_clean
);

Running query in 'duckdb:///data/vermont.duckdb'

num_group_checklists_missing_from_dask
40807


In [39]:
%%sql
SELECT COUNT(*) as num_group_checklists_missing_from_duckdb
FROM (
    SELECT group_id
    FROM sightings_clean
    EXCEPT
    SELECT DISTINCT "GROUP IDENTIFIER"
    FROM dask_sightings_clean
);

Running query in 'duckdb:///data/vermont.duckdb'

num_group_checklists_missing_from_duckdb
40807


In [40]:
%%sql

-- drop duplicate observations from group sightings
DROP TABLE IF EXISTS sightings_clean;
CREATE TABLE sightings_clean AS
SELECT *
FROM (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY group_id, common_name
            ORDER BY sampling_id
        ) as row_num
    FROM sightings_staging
) t
WHERE group_id IS NULL or row_num = 1

Running query in 'duckdb:///data/vermont.duckdb'

Count


In [41]:
%%sql
SELECT COUNT(*) as num_checklists
FROM (
    SELECT COUNT(*) FROM sightings_clean
    GROUP BY sampling_id
) t

Running query in 'duckdb:///data/vermont.duckdb'

num_checklists
249494
